In [0]:
/*

delete from hive_metastore.`default`.jwk_mapping_remove;

insert into hive_metastore.default.jwk_mapping_remove 
with  t_cellunavailabletime_nr_hr_silver as (
      SELECT  concat('VZ_',left(REGEXP_REPLACE(NE_NAME, '[^0-9]', ''),7)) GNODEB_ID,concat('VZ_',left(REGEXP_REPLACE(NE_NAME, '[^0-9]', ''),11)) DU_ID,replace(split(a.LOCATION,'/')[0],'CellIdentity','') GCELL_ID
				from apachedataingestion.nr.cellunavailabletime_nr_hr_silver    a
                WHERE 5=5 and  Daily  >=  date_format(date_add(now(), -10),"yyyy-MM-dd") 
				GROUP BY 1,2,3

),
t_vz_conf_gnode_du_gcell_silver as (
select  distinct substring(DU_ID,1,10) GNODEB_ID,DU_ID,GCELL_ID
                  		from hive_metastore.`default`.vz_conf_gnode_du_gcell_silver  where not(length(GNODEB_ID) < 10 or length(du_id) < 14) 

)
select 
COALESCE(a.GNODEB_ID,b.GNODEB_ID) gNB_ID,
COALESCE(a.DU_ID,b.DU_ID) DU_ID,
COALESCE(a.GCELL_ID,b.GCELL_ID) CellIdentity
 from t_vz_conf_gnode_du_gcell_silver a 
full join t_cellunavailabletime_nr_hr_silver b on a.GNODEB_ID=b.GNODEB_ID and a.DU_ID=b.DU_ID and a.GCELL_ID=b.GCELL_ID
where 
a.GNODEB_ID is null  or
b.GNODEB_ID  is null or 
a.DU_ID  is null     or 
b.DU_ID  is null     or 
a.GCELL_ID  is null  or
b.GCELL_ID  is null;

*/




--delete from hive_metastore.`default`.jwk_mapping_live;

merge into hive_metastore.`default`.jwk_mapping_live as gold
using ( 
        --create table  hive_metastore.`default`.jwk_mapping_live as
        select distinct case when length(gNB_ID)=6 then concat('0',gNB_ID) else gNB_ID end gNB_ID ,case when length(du_id)=10 then concat('0',DU_ID) else DU_ID end du_id ,CellIdentity from (
            /*select replace(split(LOCATION,'/')[0],'gNB_ID','') gNB_ID,replace(split(LOCATION,'/')[1],'gNB_DU_ID','') DU_ID,replace(split(LOCATION,'/')[2],'CellIdentity','') CellIdentity 
                --from hive_metastore.`default`.jwk_mapping where tb like '%cucpcslpergnbwithduidentifierhexabronze%'
                from usm.nr.cucpcslpergnbwithduidentifierhexabronze  where init_time  =  date_format(date_add(now(), {{ days }}),"yyyy-MM-dd")  and hour(init_time)=0
            union all */
            select replace(split(LOCATION,'/')[0],'gNB_ID','') gNB_ID,replace(split(LOCATION,'/')[1],'gNB_DU_ID','') DU_ID,replace(split(LOCATION,'/')[2],'CellIdentity','') CellIdentity                 
                --from usm.nr.intersnpscellchangepergnbwithduidentifierbronze  where init_time  =  date_format(date_add(now(), {{ days }}),"yyyy-MM-dd")  and hour(init_time)=0
                from apachedataingestion.nr.intersnpscellchangepergnbwithduidentifier_nr_hr_silver  where Daily  =  date_format(date_add(now(), {{ days }}),"yyyy-MM-dd")  and hourly=0                 
            union all
            select replace(split(LOCATION,'/')[0],'gNB_ID','') gNB_ID,replace(split(LOCATION,'/')[1],'gNB_DU_ID','') DU_ID,replace(split(LOCATION,'/')[2],'CellIdentityWithInvalid','') CellIdentity                 
                --from usm.nr.sgnbadditionpergnbwithduidentifierbronze  where init_time  =  date_format(date_add(now(), {{ days }}),"yyyy-MM-dd")  and hour(init_time)=0
                from apachedataingestion.nr.sgnbadditionpergnbwithduidentifier_hr_silver  where Daily  =  date_format(date_add(now(), {{ days }}),"yyyy-MM-dd")  and hourly=0
            union all
            select replace(split(LOCATION,'/')[1],'gNB_ID','') gNB_ID,replace(split(LOCATION,'/')[2],'gNB_DU_ID','') DU_ID,replace(split(LOCATION,'/')[3],'CellIdentity','') CellIdentity                 
                --from usm.nr.ueconnectionnumberpergnbwithduidentifierbronze  where init_time  =  date_format(date_add(now(), {{ days }}),"yyyy-MM-dd")  and hour(init_time)=0
                from apachedataingestion.nr.ueconnectionnumberpergnbwithduidentifier_nr_hr_silver  where Daily  =  date_format(date_add(now(), {{ days }}),"yyyy-MM-dd")  and hourly=0
            union all
            select replace(split(LOCATION,'/')[0],'gNB_ID','') gNB_ID,replace(split(LOCATION,'/')[1],'gNB_DU_ID','') DU_ID,replace(split(LOCATION,'/')[2],'CellIdentity','') CellIdentity                 
                --from usm.nr.intrasnpscellchangepergnbwithduidentifierbronze  where init_time  =  date_format(date_add(now(), {{ days }}),"yyyy-MM-dd")  and hour(init_time)=0
                from apachedataingestion.nr.intrasnpscellchangepergnbwithduidentifier_nr_hr_silver  where Daily  =  date_format(date_add(now(), {{ days }}),"yyyy-MM-dd")  and hourly=0
            union all
            select replace(split(LOCATION,'/')[0],'gNB_ID','') gNB_ID,replace(split(LOCATION,'/')[1],'gNB_DU_ID','') DU_ID,replace(split(LOCATION,'/')[2],'CellIdentity','') CellIdentity                 
                --from usm.nr.erabadditionpergnbwithduidentifierbronze  where init_time  =  date_format(date_add(now(), {{ days }}),"yyyy-MM-dd")  and hour(init_time)=0
                from apachedataingestion.nr.erabadditionpergnbwithduidentifier_hr_silver  where Daily  =  date_format(date_add(now(), {{ days }}),"yyyy-MM-dd")  and hourly=0
                
        ) x
) as bronze
on gold.gNB_ID=bronze.gNB_ID and gold.DU_ID=bronze.DU_ID and gold.CellIdentity=bronze.CellIdentity
when matched then update set *
when not matched then insert *;

-- select gNB_ID,DU_ID,CellIdentity from hive_metastore.`default`.jwk_mapping_live;
-- select length(gNB_ID),count(*) from hive_metastore.`default`.jwk_mapping_live group by length(gNB_ID);

delete 
-- select *
from hive_metastore.`default`.jwk_mapping_live where length(du_id) < 10;




delete from hive_metastore.default.jwk_mapping_live_4remove;

with aaa (
select GNODEB_ID,GCELL_ID,count(distinct DU_ID) from (select gNB_ID GNODEB_ID,CellIdentity GCELL_ID,DU_ID from hive_metastore.`default`.jwk_mapping_live) group by GNODEB_ID,GCELL_ID having count(distinct DU_ID)>1 -- 2 retruns
),
bbb (
select a.GNODEB_ID,a.DU_ID,a.GCELL_ID from (select gNB_ID GNODEB_ID,CellIdentity GCELL_ID,DU_ID from hive_metastore.`default`.jwk_mapping_live) a inner join aaa b on a.GNODEB_ID=b.GNODEB_ID and a.GCELL_ID=b.GCELL_ID   -- comfare to conf
)
insert into hive_metastore.default.jwk_mapping_live_4remove
select DU_ID,GCELL_ID from bbb where (DU_ID,GCELL_ID) not in (
                        select b.DU_ID,b.GCELL_ID
                        from apachedataingestion.nr.cellunavailabletime_nr_hr_silver a
                        inner join bbb b on left(REGEXP_REPLACE(NE_NAME, '[^0-9]', ''),11)=b.DU_ID and replace(a.LOCATION,'CellIdentity','')=b.gcell_id
                        where Daily  >=  date_format(date_add(now(), -1),"yyyy-MM-dd") 
                        group by b.DU_ID,b.GCELL_ID
);
/*
select * from hive_metastore.default.jwk_mapping_live_4remove where GCELL_ID='995';
select * from hive_metastore.default.jwk_mapping_live where gNB_id= '1260129' and CellIdentity='995';
*/


delete 
from hive_metastore.default.jwk_mapping_live A where EXISTS (
        select DU_ID,GCELL_ID from hive_metastore.default.jwk_mapping_live_4remove WHERE A.DU_ID = DU_ID AND A.CellIdentity = GCELL_ID
 );
